In [19]:
import src.classifiers.linear as linear
import numpy as np

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
results = linear.main('./data/pair_attacks.npz')

Data loaded: 312
Hyperparam optimization...


  6%|████████▌                                                                                                                                                | 1/18 [00:05<01:39,  5.85s/it]

new best:  0.5 l1 0.001


/home/rob/Data/JACK_HACKATHON/hackathon/myenv/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
 11%|█████████████████                                                                                                                                        | 2/18 [00:23<03:22, 12.67s/it]

new best:  1.0 l1 0.01


/home/rob/Data/JACK_HACKATHON/hackathon/myenv/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
 17%|█████████████████████████▌                                                                                                                               | 3/18 [00:40<03:44, 14.95s/it]/home/rob/Data/JACK_HACKATHON/hackathon/myenv/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
 22%|██████████████████████████████████                                                                                                                       | 4/18 [00:59<03:46, 16.18s/it]/home/rob/Data/JACK_HACKATHON/hackathon/myenv/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warning

(LogisticRegression(C=0.01, class_weight='balanced', n_jobs=-1, penalty='l1',
                    solver='saga'),
 1.0,
 {('l1', 0.001): 0.5,
  ('l1', 0.01): 1.0,
  ('l1', 0.1): 1.0,
  ('l1', 1): 1.0,
  ('l1', 10): 1.0,
  ('l1', 100): 1.0,
  ('l2', 0.001): 1.0,
  ('l2', 0.01): 1.0,
  ('l2', 0.1): 1.0,
  ('l2', 1): 1.0,
  ('l2', 10): 1.0,
  ('l2', 100): 1.0,
  ('elasticnet', 0.001): 0.5,
  ('elasticnet', 0.01): 1.0,
  ('elasticnet', 0.1): 1.0,
  ('elasticnet', 1): 1.0,
  ('elasticnet', 10): 1.0,
  ('elasticnet', 100): 1.0})

In [16]:
coef = results[0].coef_

In [22]:
coef.shape

(1, 65536)

In [56]:
topk_idx = np.argsort(np.abs(coef)[0])[::-1][:10]
topk_vals = coef[0,topk_idx]
needed_idxs = coef.nonzero()[1]

In [ ]:
import time
import pandas as pd
# load mapping
import goodfire
from tqdm import tqdm
client = goodfire.Client(api_key=api_key)

df = pd.read_csv('./feature_indexes_to_ids_map.csv')

def get_uuid_from_index(idx, model='meta-llama/Meta-Llama-3-8B-Instruct'):
    # check that `case` matches model, and index_in_sae matches idx, then return the `id` column
    return df[(df['case'] == model) & (df['index_in_sae'] == idx)]['id'].values[0]

def get_labels_from_indexs(idxs, batch_size=100):
    uuids = [get_uuid_from_index(idx) for idx in idxs]
    batched_uuids = [(uuids[i:i+batch_size], idxs[i:i+batch_size]) for i in range(0, len(uuids), batch_size)]
    labels = {}
    for batch_uuids, batch_idxs in tqdm(batched_uuids):
        features = client.features.list(batch_uuids)
        time.sleep(0.5)
        for idx, v in zip(batch_idxs, features._features.values()):
            labels[idx] = v.label
    return labels

labels = get_labels_from_indexs(needed_idxs)

  0%|                                                                                                                                                                  | 0/1 [00:00<?, ?it/s]


AttributeError: 'FeatureGroup' object has no attribute 'values'

In [72]:
df

,index_in_sae,id,case
0,0,3ab218be-1b9a-4cb6-9f7f-c0935c4ba24e,meta-llama/Meta-Llama-3-8B-Instruct
1,1,3ff807a0-bfb6-4b4d-96d5-3a34dc034e0e,meta-llama/Meta-Llama-3-8B-Instruct
2,2,865c2506-f1fa-4d67-92e5-d29da1dc28f7,meta-llama/Meta-Llama-3-8B-Instruct
3,3,ea15cfea-cecc-4944-843b-928a0b990668,meta-llama/Meta-Llama-3-8B-Instruct
4,4,33bef3ce-2ba0-42a8-950a-5c6218d68421,meta-llama/Meta-Llama-3-8B-Instruct
...,...,...,...
131067,65531,028416ae-8535-4569-be8c-2dbfb923eacc,meta-llama/Meta-Llama-3.1-70B-Instruct
131068,65532,9bb6aa2e-baf6-4d75-8b5f-f4ba7372ad2c,meta-llama/Meta-Llama-3.1-70B-Instruct
131069,65533,f8d5b0c0-4f14-49b4-a1c2-5b7702de0b8a,meta-llama/Meta-Llama-3.1-70B-Instruct
131070,65534,ecc4a017-4e7f-4f9d-ba0b-b0b5d5d59fc8,meta-llama/Meta-Llama-3.1-70B-Instruct


In [32]:
topk_idx

array([17770, 26989, 41459, 29882, 51506,  8653, 43824, 47149, 13674,
       54472])

In [50]:
# load Neel nanda pile 10k
from datasets import load_dataset

ds = load_dataset('NeelNanda/pile-10k')

README.md:   0%|          | 0.00/373 [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/921 [00:00<?, ?B/s]

In [ ]:
resp = client.features.search(ds['train'][0]['text'][:100])

(FeatureGroup([
    0: "Simple indie or casual mobile game concepts",
    1: "Interactive games and quizzes",
    2: "Game over or win condition detection",
    3: "Game play moves and actions",
    4: "The model should complete a code snippet for a simple game",
    5: "The model should complete a code snippet for the game Pong",
    6: "Action of submitting or turning in content",
    7: "Adventure and survival game themes",
    8: "Child-oriented mobile apps and games",
    9: "Use of 'that' to introduce subordinate clauses or additional information"
 ]),
 [0.4608382284641266,
  0.4347690939903259,
  0.4068910777568817,
  0.4028874337673187,
  0.39343610405921936,
  0.392151415348053,
  0.39035332202911377,
  0.37867432832717896,
  0.37514179944992065,
  0.36837878823280334])

In [80]:
client.features.list(['56b9533a-fb87-43e9-8aa3-f5da292be286'])._features.values()

odict_values([Feature("Simple indie or casual mobile game concepts")])